# Step 2: Model Building
Using the sensor data sets explored and constructed in the `1_data_ingestion_and_preparation.ipynb` Jupyter notebook, this notebook loads the data from the Azure Blob container and builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict remaining useful life of aircraft engines. We then store the model for deployment in an Azure web service. We will prepare and build the web service in the `Code/3_operationalization.ipynb` Jupyter notebook.

In [1]:
import keras

Using TensorFlow backend.


In [2]:
# import the libraries
import h5py
import os
import pandas as pd
import numpy as np

import urllib
import glob
import pickle
import re

# Setup the pyspark environment
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
spark = SparkSession.builder.getOrCreate()

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from sklearn import datasets
from keras.layers import Dense, Dropout, LSTM, Activation

# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger

# For Azure blob storage access
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

In [3]:
from azureml.logging import get_azureml_logger
run_logger = get_azureml_logger()
run_logger.log('amlrealworld.predictivemaintenanceforpm.modelbuildingandevaluation','true')

## Load feature data set

We have previously created the labeled data set in the `Code\1_Data Ingestion and Preparation.ipynb` Jupyter notebook. Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need your credentials here again.

In [4]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "<your blob storage account key>"
#-------------------------------------------------------------------------------------------
# The data from the Data Ingestion and Preparation notebook is stored in the sensordata ingestion container.
CONTAINER_NAME = "sensordataingestionpm"

# Connect to your blob service     
az_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# This is the final feature data file.
TRAIN_DATA = 'PM_train_files.parquet'
TEST_DATA = 'PM_test_files.parquet'

Load the data and dump a short summary of the resulting DataFrame.

In [5]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TRAIN_DATA):
    os.makedirs(TRAIN_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TRAIN_DATA in blob.name:
        local_file = os.path.join(TRAIN_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

train_df = spark.read.parquet(TRAIN_DATA)
train_df = train_df.toPandas()
train_df.head(10)

DONE creating a local directory!


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,RUL,label1,label2,cycle_norm
0,52,119,0.643678,0.083333,0.0,0.0,0.605422,0.424024,0.482613,0.0,...,0.0,0.500000,0.0,0.0,0.472868,0.453604,94,0,0,0.326870
1,52,120,0.189655,0.833333,0.0,0.0,0.409639,0.440811,0.392302,0.0,...,0.0,0.416667,0.0,0.0,0.527132,0.530793,93,0,0,0.329640
2,52,121,0.500000,0.750000,0.0,0.0,0.572289,0.507739,0.597907,0.0,...,0.0,0.500000,0.0,0.0,0.449612,0.544187,92,0,0,0.332410
3,52,122,0.517241,0.333333,0.0,0.0,0.466867,0.432963,0.492404,0.0,...,0.0,0.416667,0.0,0.0,0.387597,0.603010,91,0,0,0.335180
4,52,123,0.689655,0.750000,0.0,0.0,0.400602,0.468062,0.501519,0.0,...,0.0,0.250000,0.0,0.0,0.496124,0.540182,90,0,0,0.337950
5,52,124,0.747126,0.666667,0.0,0.0,0.484940,0.384783,0.493079,0.0,...,0.0,0.416667,0.0,0.0,0.589147,0.502900,89,0,0,0.340720
6,52,125,0.505747,0.750000,0.0,0.0,0.406627,0.349248,0.415429,0.0,...,0.0,0.500000,0.0,0.0,0.565891,0.628556,88,0,0,0.343490
7,52,126,0.448276,0.333333,0.0,0.0,0.463855,0.325921,0.455773,0.0,...,0.0,0.583333,0.0,0.0,0.410853,0.552886,87,0,0,0.346260
8,52,127,0.362069,0.083333,0.0,0.0,0.620482,0.373229,0.513504,0.0,...,0.0,0.416667,0.0,0.0,0.457364,0.551505,86,0,0,0.349030
9,52,128,0.500000,0.166667,0.0,0.0,0.379518,0.369087,0.392640,0.0,...,0.0,0.500000,0.0,0.0,0.527132,0.524855,85,0,0,0.351801


In [6]:
type(train_df)

pandas.core.frame.DataFrame

In [7]:
# load the previous created final dataset into the workspace
# create a local path where we store results
if not os.path.exists(TEST_DATA):
    os.makedirs(TEST_DATA)
    print('DONE creating a local directory!')

# download the entire parquet result folder to local path for a new run 
for blob in az_blob_service.list_blobs(CONTAINER_NAME):
    if TEST_DATA in blob.name:
        local_file = os.path.join(TEST_DATA, os.path.basename(blob.name))
        az_blob_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)
        
test_df = spark.read.parquet(TEST_DATA)
test_df = test_df.toPandas()
test_df.head(10)

DONE creating a local directory!


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,49,183,0.500000,0.333333,0.0,0.0,0.334337,0.430783,0.325625,0.0,...,0.0,0.333333,0.0,0.0,0.511628,0.737365,0.504155,141,0,0
1,49,184,0.436782,0.916667,0.0,0.0,0.331325,0.378897,0.573093,0.0,...,0.0,0.333333,0.0,0.0,0.596899,0.736537,0.506925,140,0,0
2,49,185,0.448276,0.916667,0.0,0.0,0.439759,0.311969,0.335584,0.0,...,0.0,0.333333,0.0,0.0,0.612403,0.772991,0.509695,139,0,0
3,49,186,0.609195,0.416667,0.0,0.0,0.340361,0.235448,0.253545,0.0,...,0.0,0.416667,0.0,0.0,0.550388,0.649268,0.512465,138,0,0
4,49,187,0.448276,0.416667,0.0,0.0,0.400602,0.268367,0.325625,0.0,...,0.0,0.583333,0.0,0.0,0.565891,0.698840,0.515235,137,0,0
5,49,188,0.666667,0.583333,0.0,0.0,0.319277,0.426641,0.324949,0.0,...,0.0,0.583333,0.0,0.0,0.596899,0.653963,0.518006,136,0,0
6,49,189,0.471264,0.166667,0.0,0.0,0.427711,0.308044,0.225861,0.0,...,0.0,0.250000,0.0,0.0,0.620155,0.698288,0.520776,135,0,0
7,49,190,0.718391,0.166667,0.0,0.0,0.487952,0.390669,0.367995,0.0,...,0.0,0.333333,0.0,0.0,0.573643,0.588373,0.523546,134,0,0
8,49,191,0.431034,0.416667,0.0,0.0,0.219880,0.370395,0.285618,0.0,...,0.0,0.500000,0.0,0.0,0.604651,0.605220,0.526316,133,0,0
9,49,192,0.442529,0.416667,0.0,0.0,0.358434,0.235884,0.410702,0.0,...,0.0,0.333333,0.0,0.0,0.612403,0.743303,0.529086,132,0,0


In [8]:
type(test_df)

pandas.core.frame.DataFrame

## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering which is manual construction of right features using domain expertise and similar methods. This usually makes these models hard to reuse since feature engineering is specific to the problem scenario and the available data which varies from one business to the other. Perhaps the most attractive part of applying deep learning in the predictive maintenance domain is the fact that these networks can automatically extract the right features from the data, eliminating the need for manual feature engineering.

When using LSTMs in the time-series domain, one important parameter to pick is the sequence length which is the window for LSTMs to look back. This may be viewed as similar to picking window_size = 5 cycles for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) which are rolling mean and rolling standard deviation for 21 sensor values. The idea of using LSTMs is to let the model extract abstract features out of the sequence of sensor values in the window rather than engineering those manually. The expectation is that if there is a pattern in these sensor values within the window prior to failure, the pattern should be encoded by the LSTM.

One critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. For example, computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing and abstracting of values over such a long period, istead, using all 50 values as input may provide better results. While feature engineering over large window sizes may not make sense, LSTMs are able to use larger window sizes and use all the information in the window as input. Below, we illustrate the approach.

In [9]:
# pick a large window size of 50 cycles
sequence_length = 50

[Keras LSTM](https://keras.io/layers/recurrent/) layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step. 

In [10]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [11]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
input_features = sensor_cols + sequence_cols
input_features
sequence_cols.extend(sensor_cols)

In [14]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())


In [15]:
# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(15631, 50, 25)

In [16]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [17]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['label1']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(15631, 1)

## LSTM Network

Next, we build a deep network. The first layer is a LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

In [18]:
# build the network
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()

model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=nb_out, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 100)           50400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 80,651
Trainable params: 80,651
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
%%time
# fit the network
model.fit(seq_array, label_array, epochs=10, batch_size=200, validation_split=0.05, verbose=1,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')])

Train on 14849 samples, validate on 782 samples
Epoch 1/10
14849/14849 [==============================] - 17s 1ms/step - loss: 0.2624 - acc: 0.8895 - val_loss: 0.1817 - val_acc: 0.9105
Epoch 2/10
14849/14849 [==============================] - 16s 1ms/step - loss: 0.1076 - acc: 0.9572 - val_loss: 0.0727 - val_acc: 0.9757
Epoch 3/10
14849/14849 [==============================] - 16s 1ms/step - loss: 0.0914 - acc: 0.9634 - val_loss: 0.0977 - val_acc: 0.9591
CPU times: user 3min, sys: 48.1 s, total: 3min 48s
Wall time: 50.4 s


In [21]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Accurracy: {}'.format(scores[1]))

15631/15631 [==============================] - 5s 342us/step
Accurracy: 0.9485637493573135


In [22]:
# make predictions and compute confusion matrix
y_pred = model.predict_classes(seq_array,verbose=1, batch_size=200)
y_true = label_array
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

15631/15631 [==============================] - 5s 348us/step
Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[12503,    37],
       [  767,  2324]])

In [23]:
# compute precision and recall
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print( 'precision = ', precision, '\n', 'recall = ', recall)

precision =  0.9843286742905548 
 recall =  0.7518602394047234


Next, we look at the performance on the test data. Only the last cycle data for each engine id in the test data is kept for testing purposes. In order to compare the results to the template, we pick the last sequence for each id in the test data.

In [24]:
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
seq_array_test_last.shape

(93, 50, 25)

In [25]:
y_mask = [len(test_df[test_df['id']==id]) >= sequence_length for id in test_df['id'].unique()]

In [26]:
label_array_test_last = test_df.groupby('id')['label1'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)
label_array_test_last.shape

(93, 1)

In [27]:
print(seq_array_test_last.shape)
print(label_array_test_last.shape)

(93, 50, 25)
(93, 1)


In [28]:
# test metrics
scores_test = model.evaluate(seq_array_test_last, label_array_test_last, verbose=2)
print('Accurracy: {}'.format(scores_test[1]))

Accurracy: 0.6881720398062019


In [29]:
# make predictions and compute confusion matrix
y_pred_test = model.predict_classes(seq_array_test_last)
y_true_test = label_array_test_last
print('Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true_test, y_pred_test)
cm

Confusion matrix
- x-axis is true labels.
- y-axis is predicted labels


array([[61, 11],
       [18,  3]])

In [30]:
# compute precision and recall
precision_test = precision_score(y_true_test, y_pred_test)
recall_test = recall_score(y_true_test, y_pred_test)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Precision: ', precision_test, '\n', 'Recall: ', recall_test,'\n', 'F1-score:', f1_test )

Precision:  0.21428571428571427 
 Recall:  0.14285714285714285 
 F1-score: 0.17142857142857143


## Saving/loading the model  

In [31]:
# saving a copy of the file
model_cp = model

In [32]:
model_cp

### Save and test the loaded model locally first

In [33]:
# Save the model for operationalization: https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import os
import h5py
from sklearn import datasets 
 
# save model
# serialize model to JSON
model_json = model.to_json()
with open("modellstm.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modellstm.h5")
print("Model saved")

Model saved


In [34]:
from keras.models import model_from_json

# load json and create model
json_file = open('modellstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("modellstm.h5")
print("Model loaded")

Model loaded


In [35]:
score = loaded_model.predict_proba(seq_array,verbose=1)
print(score.shape)
print(score)

15631/15631 [==============================] - 10s 645us/step
(15631, 1)
[[0.02161   ]
 [0.0240796 ]
 [0.02850101]
 ...
 [0.9828591 ]
 [0.98365027]
 [0.9840912 ]]


### Save the files to the shared folder for operationalization
https://docs.microsoft.com/en-us/azure/machine-learning/preview/how-to-read-write-files

#### Ubuntu Linux
/home/<username>/ then navigate to .azureml/share/<exp_acct_name>/<workspace_name>/<proj_name>/

In [36]:
with open(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'] + 'modellstm.json', 'wt') as json_file:
    json_file.write(model_json)
    print("json file written shared folder")
    json_file.close()

json file written shared folder


In [37]:
model_cp.save_weights(os.path.join(os.environ['AZUREML_NATIVE_SHARE_DIRECTORY'], 'modellstm.h5'))

## Conclusion
In the next notebook `Code/3_operationalization.ipynb` Jupyter notebook we will create the functions needed to operationalize and deploy any model to get realtime predictions.